In [ ]:
import numpy as np
import os
from matplotlib import pyplot as plt
import glob
from collections import Counter
from sklearn.linear_model import LinearRegression
from lmfit.models import Model
import re
from sklearn.model_selection import train_test_split

import pyconll

from tqdm.notebook import tqdm

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

## UD to txt

In [ ]:
my_conll_file_location = 'hy_bsut-ud-train.conllu'
train = pyconll.load_from_file(my_conll_file_location)

In [ ]:
with open('ud_eastern.txt', 'w') as f:
    for sentence in train:
        # Do work within loops
        f.write(sentence.text + '\n')

## Stop-words

In [ ]:
def preprocess(sentence):
    # punctuation=['.','-',',','!','?','(','—',')','՞','՛','։','՝','՜','’','«','»','*','\n','=',':','[',']','/',';','․','`','\t','%','$','\xa0','\r','_','●','0','1','2','3','4','5','6','7','8','9']
    punctuation = ['՜', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '-', '—', '։']

    for spaced in punctuation:
        sentence = sentence.replace(spaced, '').lower()

    sentence = re.sub(" +", " ", sentence)

    txt = sentence.replace('\n', '').lower()
    txt = txt.split(' ')
    txt = [t for t in txt if t != '']
    return txt

In [ ]:
%%time
folders = ['eastern', 'western', 'grabar']
worlds_list_dict = {}

for folder in folders:
    files_paths = glob.glob(folder + '/*.txt')
    names = [path.replace('/', ' ')[:15] for path in files_paths]
    files = [' '.join(open(path, 'r', encoding='utf8').readlines()) for path in files_paths]

    words_list = []
    for sentence in files:
        words_list.extend(preprocess(sentence))

    print(words_list.__len__())
    worlds_list_dict[folder] = Counter(words_list)


In [ ]:
with open('stop_eastern.txt', 'w') as f:
    for i, line in enumerate(worlds_list_dict['eastern'].most_common()):
        if line[1] >= 150:
            f.write(str(line[0]) + '\n')
        else:
            break

## Load and split data

In [ ]:
%%time
folders = ['eastern', 'western', 'grabar']
files_dict = {}

for folder in folders:
    files_paths = glob.glob(folder + '/*.txt')
    files = [' '.join(open(path, 'r', encoding='utf8').readlines()) for path in files_paths]
    files_dict[folder] = files

In [ ]:
%%time

preprocess_files_dict = {}
batch_len = 20

for folder in folders:
    files = files_dict[folder]
    batches_list = []

    for file in files:
        words = preprocess(file)
        batches = np.array_split(words, np.ceil(words.__len__() / batch_len))
        batches_list.extend(batches)

    preprocess_files_dict[folder] = batches_list


In [ ]:
dataset_text = []
dataset_labels = []

for i, folder in enumerate(folders):
    text_batches = preprocess_files_dict[folder]
    dataset_text.extend(text_batches)
    dataset_labels.extend(np.full(len(text_batches), i))

## Lexical (stop-words) descriptors

In [ ]:
western_stop = ['ենք', 'էի', 'թ', 'ին', 'մենք', 'որոնք', 'պիտի', 'և', 'որպեսզի', 'վրայ', 'կ՚', 'կը', 'մը', 'մըն',
                'անոր', 'ալ', 'ան', 'քեզ', 'եթէ', 'թէ', 'որպէս']

grabar_stop = ['դու', 'եք', 'ըստ', 'նա', 'պիտի', 'վրայ', 'զի', 'ընդ', 'քո', 'քեզ', 'եթէ', 'թէ', 'որպէս']

eastern_stop = ['դու', 'ենք', 'եք', 'էի', 'ըստ', 'ին', 'հետո', 'մենք', 'մեջ', 'նա', 'նաև', 'նրա', 'նրանք', 'որը',
                'որոնք', 'որպես', 'ում', 'վրա', 'և', 'որպեսզի']

western_stop = set(western_stop)
grabar_stop = set(grabar_stop)
eastern_stop = set(eastern_stop)

In [ ]:
def get_lexical_desc(words):
    intersect_western = len(set(words) & western_stop) / len(western_stop)
    intersect_grabar = len(set(words) & grabar_stop) / len(grabar_stop)
    intersect_eastern = len(set(words) & eastern_stop) // len(eastern_stop)

    return intersect_western, intersect_grabar, intersect_eastern

## Morphemic descriptors

In [ ]:
grabar_suffixes = ['աւք', 'եալ', 'եան', 'իւք', 'ոյց', 'ովք', 'ուց', 'ուցան']
grabar_prefixes = ['ապա', 'արտ', 'բաղ', 'բաղա', 'դեր', 'ենթ', 'ենթա', 'ընթա', ' համ', 'համա', 'հան', 'հոմ', 'հոմա',
                   'տար', 'տարա']

eastern_suffixes = ['աբար', 'ագին', 'ագույն', 'ածո', 'ածու', 'ական', 'ակերտ', 'ային', 'անակ', 'անի', 'անոց', 'անք',
                    'ապան', 'ապանակ', 'ապատ', 'ապես', 'աստան', 'ավետ', 'ավուն', 'արան', 'արար', 'արեն', 'արք', 'ացի',
                    'ացն-', 'ացու', 'բան', 'բար', 'գին', 'գույն', 'եղեն', 'ենի', 'երեն', 'երորդ', 'եցն-', 'լիկ', 'կերտ',
                    'կոտ', 'մունք ', 'յալ', 'յակ', 'յան', 'յանց', 'յուն նախա-', 'ներ', 'նոց', 'ոնք', 'ովին', 'որդ',
                    'որեն', 'ոցի', 'ուք', 'պան', 'պանակ', 'ստան', 'ված', 'վածք', 'ավոր', 'վոր', 'ություն', 'ուլ', 'ուկ',
                    'ուհի', 'ում', 'ույթ', 'ույր', 'ուն', 'ուտ', 'ուրդ', 'ուց']
eastern_prefixes = ['ամենա', 'այսր', 'անդր', 'ապա', 'ավտո', 'արտ', 'արտա', 'բենզա', ', գեր', 'գերա', 'դեր', 'ենթա',
                    'եվրա', ' էլեկտրա', 'թեր', 'թերա', 'կենս', 'կինո', 'հակ', 'հակա', 'համ', 'համա', 'հար', 'հարա',
                    'հեռա', 'հեռուստա', 'հոմա', 'մակ', 'մակրո', 'միկրո', 'միջ', 'նախ', 'ներ', 'ստոր', 'վեր', 'վերա',
                    'տար', 'տարա', 'փոխ', 'քառ', 'քառա']

western_reform = ['իլ', 'իուն', 'եան', 'յ', 'օ', 'է', 'յ', 'վո', 'ոյ', 'եա', 'եօ', 'իւ', 'ու', 'ւ,' 'յե', 'եյ', 'զի',
                  'եւ', 'ել', 'յուն', 'յան', 'ում', 'ո', 'ե', 'հ', 'ո', 'ույ', 'յա', 'յո', 'յու', 'վ', 'ե', ]

## Create dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

## Create RTF model

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)

In [ ]:
print(clf.predict([[0, 0, 0, 0]]))